In [1]:
import sys
sys.path.append('..')

from src.pipeline import *

import sqlite3

conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# Convert timestamp from JST to UTC (-09:00) for all records
cursor.execute("UPDATE metadata SET created_at = datetime(created_at, '-9 hours')")
conn.commit()


c:\Users\hiroga\miniconda3\envs\pokemon-pal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# insert nobg job to SQLite manually.
from datetime import datetime
from src.pipeline import Step, create_job

# Insert a nobg job manually
create_job(Step.nobg, datetime.utcnow())


In [3]:
# Remove all nobg and crop job, except the latest.
import sys
sys.path.append('..')

from src.pipeline import *

for step in [Step.nobg, Step.cropped]:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute("SELECT MAX(executed_at) FROM jobs WHERE step == ?", (step.value,))
    latest_job = cursor.fetchone()[0]

    cursor.execute("DELETE FROM jobs WHERE step == ? AND executed_at != ?", (step.value, latest_job))
    conn.commit()


In [5]:
import sys
sys.path.append('..')

from src.pipeline import *

from datetime import datetime

# 画像を保存しなおしたことに加え、クロッピングの基準を64x64から512x512に変更したので、それ以前のメタデータを削除
latest_raw_images_saved_at = datetime.strptime('2024-02-04 23:04:03.539261', '%Y-%m-%d %H:%M:%S.%f')
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

cursor.execute("DELETE FROM metadata WHERE created_at < ?", (latest_raw_images_saved_at,))
conn.commit()
